In [1]:
import os.path
import sys
sys.path.append('/Users/cmagri1/OneDrive - Johns Hopkins/git')
from torch import nn
import torch
from torchvision.models.alexnet import alexnet
from tqdm import tqdm

from EncodingModel_cm296 import utils as emutils
from EncodingModel_cm296 import feature_extractor as emfe

import pandas as pd
import numpy as np
import csv

from utilsCM import Subject
from utilsCM import cv_regression #Predicting fMRI reponses with features (Alexnet activations) 
#cv_regression computes cross-validated ridge regression. cross-validation groupings were pre-set based
#on fMRI design. 9-categories out, 9 folds. r is averaged over folds, weights are computed over all data


In [2]:
# specify variables
resolutionval = 227;
# print(model)

#sepcify directory where images (for which we have fMRI data) are
PathToImgs = 'data-object2vec/stimuli'

In [3]:
#Load word2sense
#already preprocessed in MATLAB so that Wrd2Sns and THINGs overlap --> we have IMAGES, LABELS and SENSES
pathtofile = '../code-00-preprocessdataset/'
Wrd2Sense = pd.read_csv(pathtofile + "ThingsWrd2Sns.txt", sep=",")
ImgInfo = pd.read_csv(pathtofile + "KeptTHINGSInfo.txt", sep=",")
pathtoTHINGS = '/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense//THINGSdataset/Main/images'

In [4]:
#Compute DNN features
def object_by_feature(PathToImgs,keyword,ilayer,resolutionval):
    datapath = 'data-00/'
    if os.path.isfile(datapath+ keyword + '_' + ilayer + ".csv"):
        print('loading file with '+ keyword+ ' stimuli features for ' + ilayer)
        features = pd.read_csv(datapath+ keyword + "_" + ilayer + ".csv"  , sep=",", header=None, index_col=0)
#         features = pd.read_csv(datapath+ keyword + "_" + ilayer + ".csv"  , sep=",", header=None)

        condition_features = {}
        for index, row in features.iterrows():
            condition_features[index] = row 
    else:
        print('Computing '+ keyword+ ' stimuli features for ' + ilayer)
        model = emfe.AlexNet(ilayer); 
        
        condition_features = emutils.compute_features(model, PathToImgs,resolutionval)
        features = pd.DataFrame(condition_features).transpose() 
        features.to_csv(datapath + keyword+ "_" + ilayer + ".csv", index = True, header=False)
#     layer_by_features[ilayer] = features; #create multistructure with all layers
    return condition_features

In [5]:
layer = {'conv_1','conv_2','conv_3','conv_4','conv_5','fc_1','fc_2','fc_3'};
ROI = {'EVC','LOC','PFS','OPA','PPA','RSC','FFA','OFA','STS','EBA'}
Sub = [1,2,3,4]

In [6]:
for ilayer in layer:    

     #Compute features of CNN
     #---------Object2Vect-----------------
    keyword = 'object2vec'
    object2vec_features = object_by_feature(PathToImgs,keyword,ilayer,resolutionval)
#     #---------THINGS-----------------
    keyword = 'things'
    things_features = object_by_feature(pathtoTHINGS,keyword,ilayer,resolutionval)
    things_features_df = pd.DataFrame(things_features).transpose()
    
    for iSub in Sub:
        for iROI in ROI:
            subject = Subject(iSub,[iROI])
            #predict fMRI using object2vec DNN activations
            weights, r = cv_regression(object2vec_features, subject, l2=0)
            
            #     #---Compute Predicted ROI response to THINGS dataset
            print("Computing ROI prediction for " + ilayer)    
            ROIpred = {}
            ROIpred = np.dot(things_features_df.iloc[:,:].to_numpy(),weights.transpose())
            
            np.save(os.getcwd() + '/data-00/ROIpred_Sub' + str(iSub) + '_' + iROI + "_" + ilayer, ROIpred)
print('things dataset activations in DNN '+ ilayer +' size: ' + str(things_features_df.shape))
print('weights size: ' + str(weights.shape))
print('ROI pred size: ' + str(ROIpred.shape))
print("Done!")

Computing object2vec stimuli features for conv_3


100%|██████████| 81/81 [00:19<00:00,  4.22it/s]


Computing things stimuli features for conv_3


 55%|█████▍    | 1019/1854 [09:32<08:54,  1.56it/s]

KeyboardInterrupt: 